## Oscar Dataset
* Contains 3.7 GB of nepali dataset
* [hf](https://huggingface.co/datasets/oscar-corpus/OSCAR-2201)


# the plan
```
for data in datas:
  [X] Save oscar dataset to a csv file:
    url, text
  * store oscar crawled_url in bloom function
  * url for url in to_crawl if url not in oscar_bloom_function
  * save_to_db([url for url in to_crawl? of url not in oscar_bloom_function])
```

In [6]:
!pip install datasets --quiet
!pip install pybloom-live --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 5.2 MB/s eta 0:00:00


In [3]:
%cd /content/drive/MyDrive/Research/datasets/crwled_by_others/oscar

/content/drive/MyDrive/Research/datasets/crwled_by_others/oscar


In [ ]:
from bloom import get_bloom_thread

n = 200_000_000  # expected number of elements
p = 0.000001       # false positive probability
save_file = 'bloom_filter.pkl'

bloom_filter = get_bloom_thread(n, p, save_file)
bloom_filter.add('123')
print('123' in bloom_filter)
bloom_filter.size()

		 Creaetd new bloom filter

True


45

In [ ]:
# Function to save to csv
import csv
import os

def save_to_csv(file_path, url, text):
    # Define the headers for the CSV file
    headers = ['url', 'text']

    # Check if the file exists
    file_exists = os.path.isfile(file_path)

    # Open the file in append mode
    with open(file_path, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, escapechar='\\')

        # If the file does not exist, write the headers
        if not file_exists:
            writer.writerow(headers)

        # Write the URL and text to the CSV file
        writer.writerow([url, text])

# Example usage
# save_to_csv('data.csv', 'http://example.com', 'This is an example text.')


In [ ]:
from datasets import load_dataset
from google.colab import userdata

dataset = load_dataset("oscar-corpus/OSCAR-2201",
                        token=userdata.get('HUGGING_FACE_BEARER'), # HUGGINGFACE_API_KEY
                        language="ne",
                        streaming=True, # optional
                        trust_remote_code=True,
                        split="train") # optional, but the dataset only has a train split
n=0
for d in dataset:
    '''
     Dict with keys: ['id' <int>, 'text': <str>, 'meta']
     url = a['meta']['warc_headers']['warc-target-uri']# e.g. https://www.himalkhabar.com/news/118067
    '''
    n+=1
    url = d['meta']['warc_headers']['warc-target-uri']
    text = d['text']
    if url not in bloom_filter:
      bloom_filter.add(url)
      save_to_csv('oscar_2201_ne_data.csv', url, text.replace('|', '।'))
    if n % 10000 == 0:
      print(f'n: {n} saved:{n*10000}')

# Save bloom_filter
bloom_filter.save_bloom_filter()

n: 10000 saved:100000000
n: 20000 saved:200000000
n: 30000 saved:300000000
n: 40000 saved:400000000
n: 50000 saved:500000000
n: 60000 saved:600000000
n: 70000 saved:700000000
n: 80000 saved:800000000
n: 90000 saved:900000000
n: 100000 saved:1000000000
n: 110000 saved:1100000000
n: 120000 saved:1200000000
n: 130000 saved:1300000000
n: 140000 saved:1400000000
n: 150000 saved:1500000000
n: 160000 saved:1600000000
n: 170000 saved:1700000000
n: 180000 saved:1800000000
n: 190000 saved:1900000000
n: 200000 saved:2000000000
n: 210000 saved:2100000000
n: 220000 saved:2200000000
n: 230000 saved:2300000000
n: 240000 saved:2400000000
n: 250000 saved:2500000000
n: 260000 saved:2600000000
n: 270000 saved:2700000000
n: 280000 saved:2800000000
n: 290000 saved:2900000000
n: 300000 saved:3000000000
n: 310000 saved:3100000000
n: 320000 saved:3200000000
n: 330000 saved:3300000000
n: 340000 saved:3400000000
n: 350000 saved:3500000000
n: 360000 saved:3600000000
n: 370000 saved:3700000000
n: 380000 saved:380

In [4]:
!ls

bloom_filter.0pkl  bloom.py		   Oscar-2201-np-Dataset.ipynb
bloom_filter.pkl   oscar_2201_ne_data.csv  __pycache__


In [11]:
from bloom import BloomFilterThread
import csv
import sys

# Increase CSV field size limit
csv.field_size_limit(sys.maxsize)

# Count how many urls from oscar dataset are in bloom filter
count = 0
total_urls = 0

# load bloom filter
bloom_filter = BloomFilterThread(save_file='bloom_filter.pkl')

# Load dataset from csv file
with open('oscar_2201_ne_data.csv', 'r', encoding='utf-8', errors='replace') as csvfile:
    reader = csv.reader((line.replace('\0', '') for line in csvfile))
    next(reader, None)  # Skip the header row
    for row in reader:
        try:
            total_urls += 1
            if row:  # Check if the row is not empty
                url = row[0]
                if url in bloom_filter:
                    count += 1
            if total_urls % 10000 == 0:
                print(f'total_urls: {total_urls}, count: {count}', end='\r')
        except csv.Error as e:
            print(f"Error reading row {total_urls}: {e}")
            continue

print(f'\n\nNumber of URLs from Oscar dataset in bloom filter: {count}')
print(f'Total URLs from Oscar dataset: {total_urls}')

Bloom filter loaded from file: bloom_filter.pkl

total_urls: 400000, count: 400000

Number of URLs from Oscar dataset in bloom filter: 401606
Total URLs from Oscar dataset: 401606


In [21]:
# EC2

from bloom import BloomFilterThread

# load bloom filter
oscar_bloom_filter = BloomFilterThread(save_file='oscar_bloom_filter.pkl')

url = 'https://www.himalkhabar.com/news/118067'
url in oscar_bloom_filter

Bloom filter loaded from file: bloom_filter.pkl



True